# EdgeYolo


In [1]:
! git clone https://github.com/LSH9832/edgeyolo.git
%cd edgeyolo
! pip install -r requirements.txt

!pip install tensorrt

! git clone https://github.com/NVIDIA-AI-IOT/torch2trt.git
%cd torch2trt
! python setup.py install
%cd ..
!ls

Cloning into 'edgeyolo'...
remote: Enumerating objects: 1906, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 1906 (delta 483), reused 342 (delta 342), pack-reused 1381 (from 3)
Receiving objects: 100% (1906/1906), 62.89 MiB | 30.23 MiB/s, done.
Resolving deltas: 100% (1025/1025), done.
/kaggle/working/edgeyolo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s

## Downlaod Dataset


In [11]:
# Dataset
# https://drive.google.com/file/d/16MaG0L0M5M-ivtYKO7msq2ehTFrmGhzf/view?usp=sharing
!gdown --id 16MaG0L0M5M-ivtYKO7msq2ehTFrmGhzf
!tar -xJf /kaggle/working/edgeyolo/coco2017.tar.xz

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16MaG0L0M5M-ivtYKO7msq2ehTFrmGhzf
To: /kaggle/working/edgeyolo/coco2017.tar.xz
100%|██████████████████████████████████████| 76.9M/76.9M [00:01<00:00, 60.0MB/s]


## Train Config: train_coco.yml


In [3]:
!cat /kaggle/working/edgeyolo/params/train/train_coco.yaml

# models & weights------------------------------------------------------------------------------------------------------
model_cfg: "params/model/edgeyolo_tiny.yaml"         # model structure config file
weights: "output/train/edgeyolo_tiny_coco/last.pth"  # contains model_cfg, set null or a no-exist filename if not use it
use_cfg: false                                       # force using model_cfg instead of cfg in weights to build model

# output----------------------------------------------------------------------------------------------------------------
output_dir: "output/train/edgeyolo_tiny_coco"        # all train output file will save in this dir
save_checkpoint_for_each_epoch: true                 # save models for each epoch (epoch_xxx.pth, not only best/last.pth)
log_file: "log.txt"                                  # log file (in output_dir)

# dataset & dataloader--------------------------------------------------------------------------------------------------
dataset_cfg:

In [ ]:
import yaml

train_config = {
    "model_cfg": "params/model/edgeyolo_s.yaml",
    "weights": "output/train/edgeyolo_s_coco/last.pth",
    "use_cfg": False,
    "output_dir": "output/train/edgeyolo_s_coco",
    "save_checkpoint_for_each_epoch": True,
    "log_file": "log.txt",
    "dataset_cfg": "params/dataset/coco.yaml",
    "batch_size_per_gpu": 4,
    "loader_num_workers": 2,
    "num_threads": 1,
    "device": [0, 1],
    "fp16": False,
    "cudnn_benchmark": False,
    "optimizer": "SGD",
    "max_epoch": 25,
    "close_mosaic_epochs": 1,
    "lr_per_img": 0.00015625,
    "warmup_epochs": 5,
    "warmup_lr_ratio": 0.0,
    "final_lr_ratio": 0.05,
    "loss_use": ["bce", "bce", "giou"],
    "input_size": [640, 640],
    "multiscale_range": 5,
    "weight_decay": 0.0005,
    "momentum": 0.9,
    "enhance_mosaic": False,
    "use_ema": True,
    "enable_mixup": True,
    "mixup_scale": [0.5, 1.5],
    "mosaic_scale": [0.1, 2.0],
    "flip_prob": 0.5,
    "mosaic_prob": 1,
    "mixup_prob": 1,
    "degrees": 10,
    "hsv_gain": [0.0138, 0.664, 0.464],
    "eval_at_start": False,
    "val_conf_thres": 0.001,
    "val_nms_thres": 0.65,
    "eval_only": False,
    "obj_conf_enabled": True,
    "eval_interval": 1,
    "print_interval": 100,
    "load_optimizer_params": True,
    "train_backbone": True,
    "train_start_layers": 51,
    "force_start_epoch": -1,
}

yaml_path = "/kaggle/working/edgeyolo/params/train/train_coco.yaml"

with open(yaml_path, "w") as f:
    yaml.dump(train_config, f, default_flow_style=False)

print("✅ Train config successfully overwritten.")

✅ Train config successfully overwritten.


## Dataset Config: coco.yml


In [ ]:
!cat /kaggle/working/edgeyolo/params/dataset/coco.yaml

In [ ]:
import yaml

yaml_path = "/kaggle/working/edgeyolo/params/dataset/coco.yaml"

new_config = {
    "type": "coco",
    "dataset_path": "coco2017",
    "kwargs": {"suffix": "jpg", "use_cache": True},
    "train": {
        "image_dir": "train2017",
        "label": "annotations/instances_train2017.json",
    },
    "val": {"image_dir": "val2017", "label": "annotations/instances_val2017.json"},
    "test": {"test_dir": "test2017"},
    "segmentation_enabled": False,
    "names": [
        "0",
        "1",
        "2",
    ],
}

with open(yaml_path, "w") as f:
    yaml.dump(new_config, f, default_flow_style=False)

print("✅ YAML file successfully overwritten.")

✅ YAML file successfully overwritten.


## Training


In [25]:
!python train.py --cfg ./params/train/train_coco.yaml

20250508_113454 edgeyolo.models:97 - no weight file found, setup models from cfg file /kaggle/working/edgeyolo/params/model/edgeyolo_s.yaml
20250508_113454 edgeyolo.models.yolo:922 - Overriding models.yaml nc=80 with nc=3
20250508_113454 edgeyolo.models.yolo:922 - Overriding models.yaml nc=80 with nc=3
/kaggle/working/edgeyolo/edgeyolo/train/trainer.py:104: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.params["fp16"])
add im
add im
add im
add ia
add ia
add ia
/kaggle/working/edgeyolo/edgeyolo/train/trainer.py:104: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.params["fp16"])
20250508_113456 edgeyolo.train.trainer:334 - params:
╒════════════════════════════════╤═══════════════════════════════════════╕
│ Keywords        

# Evaluate


In [33]:
!python evaluate.py --weights output/train/edgeyolo_tiny_coco/best.pth --dataset params/dataset/coco.yaml --batch 16 --device 0 --input-size 640 640 --save

2025-05-08 12:32:28.090 | ERROR    | edgeyolo.models:__init__:107 - no weights and cfg found!
Traceback (most recent call last):
  File "/kaggle/working/edgeyolo/evaluate.py", line 211, in <module>
    launch(
  File "/kaggle/working/edgeyolo/evaluate.py", line 204, in launch
    eval_single(params=params)
  File "/kaggle/working/edgeyolo/evaluate.py", line 148, in eval_single
    ey = EdgeYOLO(weights=params.get("weights"))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/edgeyolo/edgeyolo/models/__init__.py", line 108, in __init__
    assert False
           ^^^^^
AssertionError


## Plot Config: plot.py


In [ ]:
file_path = "/kaggle/working/edgeyolo/plot.py"

with open(file_path, "r") as f:
    lines = f.readlines()

with open(file_path, "w") as f:
    for line in lines:
        if "matplotlib.use(" in line:
            f.write("matplotlib.use('Agg')\n")
        else:
            f.write(line)

In [35]:
!python plot.py --all -f output/train/edgeyolo_tiny_coco --save --format png 

2025-05-08 12:32:33.568 | INFO     | edgeyolo.utils.plot_fig:plot_all:132 - figs will be saved to output/train/edgeyolo_tiny_coco/figures, please wait for seconds and see
2025-05-08 12:32:33.568 | ERROR    | edgeyolo.utils.plot_fig:plot_all:138 - plot error: output/train/edgeyolo_tiny_coco/log.txt does not exist!


In [36]:
!cat /kaggle/working/edgeyolo/plot.py

import argparse
import os.path as osp

from edgeyolo.utils.plot_fig import plot, plot_ap, plot_all

import matplotlib
matplotlib.use('Agg')

DEFAULT_SIZE = (20, 9)
DEFAULT_SUFFIX = ["svg"]


def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("-ap", "--ap", action="store_true")
    parser.add_argument("-loss", "--loss", action="store_true")
    parser.add_argument("-lr", "--lr", action="store_true")
    parser.add_argument("-a", "--all", action="store_true")

    parser.add_argument("-f", "--file", type=str, required=True)

    parser.add_argument("-s", "--save", action="store_true")
    parser.add_argument("--format", type=str, default=DEFAULT_SUFFIX, nargs="+")
    parser.add_argument("--no-show", action="store_true")
    return parser.parse_args()


if __name__ == '__main__':
    args = get_args()
    if args.ap:
        plot_ap(args.file, show=not args.no_show, save=args.save, suffix=args.format, figsize=DEFAULT_SIZE)
    elif args.loss:
        plot(arg

## Export: ONNX


In [5]:
# custom onnx - https://drive.google.com/file/d/1brEcFBMN1rjAfA2ausSMxVRgGarJcCld/view?usp=sharing
!gdown --id 1brEcFBMN1rjAfA2ausSMxVRgGarJcCld

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1brEcFBMN1rjAfA2ausSMxVRgGarJcCld
From (redirected): https://drive.google.com/uc?id=1brEcFBMN1rjAfA2ausSMxVRgGarJcCld&confirm=t&uuid=45a4dcc6-529e-45bb-b1d8-0496f48af79e
To: /kaggle/working/edgeyolo/epoch_50.pth
100%|██████████████████████████████████████| 47.1M/47.1M [00:00<00:00, 62.9MB/s]


In [37]:
!python export.py --onnx --weights /kaggle/working/edgeyolo/output/train/edgeyolo_s_coco/best.pth --input-size 640 640 --batch 1 --opset 11 --no-simplify 

2025-05-08 12:32:39.045 | INFO     | edgeyolo.models:__init__:50 - loading models from weight /kaggle/working/edgeyolo/output/train/edgeyolo_s_coco/best.pth
/kaggle/working/edgeyolo/edgeyolo/models/__init__.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file.

In [ ]:
import torch

available_gpus = list(range(torch.cuda.device_count()))
print(f"Available GPUs: {available_gpus}")
device = available_gpus

Available GPUs: [0, 1]


In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

onnx_model_path = "/kaggle/working/edgeyolo/output/export/best/640x640_batch1.onnx"

onnx_model = onnx.load(onnx_model_path)

onnx.checker.check_model(onnx_model)
print("ONNX model is valid.")

session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dummy_input = np.random.random_sample(session.get_inputs()[0].shape).astype(np.float32)

result = session.run([output_name], {input_name: dummy_input})

print("Inference result:", result)

ONNX model is valid.
Inference result: [array([[[8.77798939e+00, 7.26758480e+00, 1.21391808e+02, ...,
         6.55770302e-03, 2.33633965e-01, 1.10857815e-01],
        [1.39679241e+01, 7.72220039e+00, 3.02407593e+02, ...,
         5.88473678e-03, 2.11204499e-01, 1.32242709e-01],
        [2.07673416e+01, 6.57302999e+00, 3.60972595e+02, ...,
         9.44903493e-03, 2.27701098e-01, 1.65245295e-01],
        ...,
        [5.50486206e+02, 6.30599243e+02, 1.52092285e+02, ...,
         6.50972128e-04, 2.74412215e-01, 1.19058251e-01],
        [5.82597656e+02, 6.31481812e+02, 1.33106110e+02, ...,
         3.17066908e-04, 2.73171961e-01, 1.00513875e-01],
        [6.12542114e+02, 6.26324097e+02, 8.09355316e+01, ...,
         7.46518373e-04, 2.75898874e-01, 7.31824636e-02]]], dtype=float32)]


In [ ]:
import numpy as np

output = result[0]
confidence_threshold = 0.1

for i in range(output.shape[0]):
    detections = output[i]
    for detection in detections:
        confidence = detection[4]
        if confidence > confidence_threshold:
            x_center, y_center, width, height = detection[:4]
            print(
                f"confidence {confidence} coordinates ({x_center}, {y_center}), "
                f"Width: {width}, Height: {height}"
            )